In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

df_ratings=pd.read_csv('./rating.csv')

reviews_groups = df_ratings.groupby("rating")["rating"].count()

plt.figure(figsize=(8, 5))
plt.bar(reviews_groups.index, reviews_groups.values, color='skyblue', edgecolor='black')
plt.title("Ratings Distribution")
plt.xlabel("Rating")
plt.ylabel("Count")
plt.xticks(reviews_groups.index)  
plt.show()